# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

- Framework는 원하는 기능 구현에 집중하여 개발할 수 있도록 일정한 형태와 필요한 기능을 갖추고 있는 골격, 뼈대
  - 이러한 뼈대 위에서 코드를 작성해 애플리케이션 개발
  - 메모리 관리, 이벤트 루프 등의 공통된 부분은 프레임워크가 관리하며, 사용자는 프레임워크가 정해준 방식대로 클래스, 메소드 구현
  - 예시
    - Java 서버 개발에 사용되는 Spring
    - Python 서버 개발에 사용되는 Django, Flask
    - 안드로이드 앱 개발에 사용되는 Android
    - 등...
- Library는 소프트웨어 개발 시, 프로그램이 사용하는 비휘발성 자원의 모임
  - 즉, 특정 기능을 모아둔 코드, 함수들의 집합
  - 예시
    - Python pip로 설치한 패키지/모듈(tensorflow, pytorch, pandas 등)
- Framework와 Library의 차이점은 "제어 흐름"의 권한이 어디에 있는가에 있다.
  - Library: 사용자가 코드의 흐름을 직접 제어해야 함
    - 개발 시 필요한 기능이 있을 경우, 능동적으로 라이브러리를 호출해 사용
  - Framework: 코드가 프레임워크게 의해 사용됨
    - 코드는 프레임워크가 짜 놓은 틀에서 수동적으로 동작

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

- 딥러닝과 머신러닝의 관계: 딥러닝은 머신러닝의 부분집합이다.
- 딥러닝과 머신러닝의 차이: 머신러닝은 자료 구조에 따라 다양한 형태를 지닌 방면, 딥러닝은 보통 전부 다 neural network로 구성
- 특징
  - 딥러닝은 머신러닝에 비해 parameter 수가 매우 많아 요구하는 데이터 수가 압도적으로 많음
  - 딥러닝은 학습시간이 매우 오래걸리고 GPU를 통해 병렬 연산 수행

### Q3. 아래의 코드에 주석 달기.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# gpu 사용이 가능한 상태(cuda가 깔리고 nvidia gpu가 존재)하는 경우 device에 gpu할당, 아니면 cpu 할당
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45) # seed 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(45) # cuda seed 고정
print(device + " is available") # 위에서 지정한 device 출력

cpu is available


In [2]:
learning_rate = 0.001 # 학습률 지정
batch_size = 100 # 배치사이즈 지정
num_classes = 10 # 클래스 수 지정
epochs = 5 # 에포크 수 지정

In [3]:
# 훈련 데이터셋 불러오기
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

# 평가 데이터셋 불러오기
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



In [4]:
# 데이터셋을 데이터로더에 얹기
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# 데이터 shape 확인
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [5]:
# 모델 정의
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # Convolution 연산 정의
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # Convolution 연산 정의
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # Dropout 정의
        self.mp = nn.MaxPool2d(2) # Max Pooling 정의
        self.fc1 = nn.Linear(320,100) # Fully-Connected Layer 정의
        self.fc2 = nn.Linear(100,10) # Fully-Connected Layer 정의


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # convolution, max pooling 연산 후 ReLU 통과
        x = F.relu(self.mp(self.conv2(x))) # convolution, max pooling 연산 후 ReLU 통과
        x = self.drop2D(x) # Dropout 적용
        x = x.view(x.size(0), -1) # 각 데이터를 1차원으로 만듦
        x = self.fc1(x) # fully connected layer 통과
        x = self.fc2(x) # fully connected layer 통과
        return F.log_softmax(x) # log softmax 적용하여 반환

In [6]:
model = ConvNet().to(device) # 모델을 device에 할당
criterion = nn.CrossEntropyLoss().to(device) # CrossEntropyLoss를 device에 할당
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Adam을 device에 할당

In [7]:
# 학습 코드
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device) # data를 device에 할당
        target = target.to(device) # target을 device에 할당
        optimizer.zero_grad() # 경사 초기화
        hypothesis = model(data) # 예측값 출력
        cost = criterion(hypothesis, target) # Loss 계산
        cost.backward() # back propagation
        optimizer.step() # parameter update
        avg_cost += cost / len(train_loader) # 평균 비용 계산
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

/var/folders/ty/d0l26xd500q6hk3ryfccwjy00000gn/T/ipykernel_1656/1780290213.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) # log softmax 적용하여 반환


[Epoch:    1]  cost = 0.338579178
[Epoch:    2]  cost = 0.114589147
[Epoch:    3]  cost = 0.089051649
[Epoch:    4]  cost = 0.076150313
[Epoch:    5]  cost = 0.0688440949


In [8]:
# 평가 코드 작성
model.eval() # 모델을 평가 모드로 수행(dropout 때문)
with torch.no_grad(): # 경사를 계산하지 않음(학습을 하지 않기 때문)
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device) # data를 device에 할당
        target = target.to(device) # target을 device에 할당
        out = model(data) # 예측값(score) 계산
        preds = torch.max(out.data, 1)[1] # 예측 class 계산
        total += len(target)
        correct += (preds==target).sum().item() # 정답 개수 계산
        
    print('Test Accuracy: ', 100.*correct/total, '%') # 정확도 출력
     

/var/folders/ty/d0l26xd500q6hk3ryfccwjy00000gn/T/ipykernel_1656/1780290213.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) # log softmax 적용하여 반환


Test Accuracy:  98.48 %


## 첫 정규세션 들으시느라 고생 많으셨습니다.